# Assignment 1 - Vaccum Agent

In this assignment, you are going to build a vaccum agent.

![](https://aimacode.github.io/aima-exercises/figures/vacuum-world-figure.png)

There are two rooms in the vaccum-cleaner world: left room A and right room B. The vaccum only can presents in one room at one time slot. The vacuum agent cleans **all** the trash in the current room if current room has trash, otherwise, the agent moves to the other room if current room is clean. The agent **never** stop cleaning or moving until time is up.

A new trash will be produced at 60% chance every time after the vaccum cleans a room or moves to another room. And then this new trash will be randomly added to either left room or right room 

Please follow the example of [BlindDog](./blindDog.ipynb) to implement following code for buidling the vacuum-clearner world.

## Agent 
**You do not need to modify anything in this section.**

In [96]:
import collections
import numbers
import random

In [97]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

In [98]:
class Agent(Thing):
    """An Agent is a subclass of Thing with one required instance attribute 
    (aka slot), .program, which should hold a function that takes one argument,
    the percept, and returns an action. (What counts as a percept or action 
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method, then the
    program could 'cheat' and look at aspects of the agent. It's not supposed
    to do that: the program can only look at the percepts. An agent program
    that needs a model of the world (and of the agent itself) will have to
    build and maintain its own model. There is an optional slot, .performance,
    which is a number giving the performance measure of the agent in its
    environment."""

    def __init__(self, program=None):
        self.alive = True
        
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

The `Agent` has one method.
* `__init__(self, program=None)`: The constructor defines various attributes of the Agent. These include

    * `alive`: which keeps track of whether the agent is alive or not 
    
    * `program`: which is the agent program and maps an agent's percepts to actions in the environment. If no implementation is provided, it defaults to asking the user to provide actions for each percept.

## ENVIRONMENT

**You do not need to modify anything in this section.**

Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [99]:

class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            print("Step", step, ":")
            
            if self.is_done():
                return
            self.step()
            
            # After each run, there is 30% chance to add a new trash at left 
            #   and another 30% chance to add a new trash at the right. 
            trash = Trash()
            random_trash = random.random()
            if random_trash < 0.3:
                self.add_thing(trash, 'right')
            elif random_trash < 0.6:
                self.add_thing(trash, 'left')

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, str):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            
            print("Added a {} at {}".format(str(thing)[1:-1], thing.location))
            
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)


`Environment` class has lot of methods! But most of them are incredibly simple, so let's see the ones we'll be using in this notebook.

* `thing_classes(self)`: Returns a static array of `Thing` sub-classes that determine what things are allowed in the environment and what aren't

* `add_thing(self, thing, location=None)`: Adds a thing to the environment at location

* `run(self, steps)`: Runs an environment with the agent in it for a given number of steps.

* `is_done(self)`: Returns true if the objective of the agent and the environment has been completed

The next two functions must be implemented by each subclasses of `Environment` for the agent to recieve percepts and execute actions 

* `percept(self, agent)`: Given an agent, this method returns a list of percepts that the agent sees at the current time

* `execute_action(self, agent, action)`: The environment reacts to an action performed by a given agent. The changes may result in agent experiencing new percepts or other elements reacting to agent input.

### ENVIRONMENT - Rooms

Rooms is an example of an environment because our vacuum can perceive and act upon it. The <b>Environment</b> class is an abstract class, so we will have to create our own subclass from it before we can use it.

Each trash is an object of class `Trash`. A single room could have more than one trash objects. And the vaccum needs to clean all the trash after executing `suck` action only once. 

Three actions are implemented in `executed_action`:

- move vaccum to left if the action is `move to left` (change the location of the vaccum).

- move vaccum to right if the action is `move to right` (change the location of the vaccum).

- clean all trash in current room if the action is `suck` (delete the Trash objects at current location).

In [100]:
class Trash(Thing):
    pass

class Rooms(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        # copy example code from BlindDog and make necessary changes. 
        # write your code here
        if action == "move to left":
            print('{} decided to {} from room {}'.format(str(agent)[1:-1], action, agent.location))
            agent.move_to_left()
        if action == "move to right":
            print('{} decided to {} from room {}'.format(str(agent)[1:-1], action, agent.location))
            agent.move_to_right()
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Trash)
            count = 1
            while items:
                if agent.clean(items[0]):
                    print('{} cleaned {} {} in room {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], count, agent.location))
                    self.delete_thing(items[0])
                    items.pop(0)
                    count += 1


### PROGRAM - Vacuum

Now that we have a <b>Rooms</b> Class, we implement our <b>Vacuum</b> to be able to move and clean.


In [101]:
class Vacuum(Agent):
    location = 'left' 
    
    def move_to_left(self):
        self.location = 'left'
        # change location of vaccum to left
        # write your code here
        # pass
        
    def move_to_right(self):
        self.location = 'right'
        # change location of vaccum to right
        # write your code here
        # pass
        
    def clean(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Trash):
            return True
        return False

Now its time to implement a <b>program</b> module for our vacuum. A program controls how the vacuum acts upon its environment. Our program will be very simple, and is shown in the table below.

Dirty means vacuum percepts trash and clean means otherwise.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Dirty</td>
        <td>Clean</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>Suck all trash</td>
       <td>Move to the other room</td>
   </tr>
        
</table>

There is only one *Thing* in the room that the vacuum agent can observe: object of the Trash. The vacuum has three actions based on the precepts:

- return action `suck` if there has trash in the room where the vaccum is at.

- return action `move to left` if the vaccum is at right room and it is clean. 

- return action `move to right` if the vacuum is at left room and it is clean.

In [102]:
# The input is percepts.
# The output is actions: suck, move to left, or move to right. 
def program(percepts):
    # print(percepts)
    # write your code here
    for p in percepts:
        if isinstance(p, Trash):
            return 'suck'
    if(p.location == 'right'):
        return 'move to left'
    if(p.location == 'left'):
        return 'move to right'

            



Run the code below. The expected output is
```
Added a Vacuum at left
Added a Trash at right
Step 0 :
Vacuum decided to move to right from room left
Added a Trash at right
Step 1 :
Vacuum cleaned Trash 1 in room right
Vacuum cleaned Trash 2 in room right
Step 2 :
Vacuum decided to move to left from room right
Step 3 :
Vacuum decided to move to right from room left
Added a Trash at right
Step 4 :
Vacuum cleaned Trash 1 in room right
Added a Trash at left
Step 5 :
Vacuum decided to move to left from room right
Added a Trash at left
Step 6 :
Vacuum cleaned Trash 1 in room left
Vacuum cleaned Trash 2 in room left
Step 7 :
Vacuum decided to move to right from room left
Step 8 :
Vacuum decided to move to left from room right
Added a Trash at right
Step 9 :
Vacuum decided to move to right from room left
Added a Trash at right
```

In [103]:
random.seed(1)

rooms = Rooms()
vacuum = Vacuum(program)
trash = Trash()
rooms.add_thing(vacuum, 'left')
rooms.add_thing(trash, 'right')

# run 10 steps
rooms.run(10)

Added a Vacuum at left
Added a Trash at right
Step 0 :
Vacuum decided to move to right from room left
Added a Trash at right
Step 1 :
Vacuum cleaned Trash 1 in room right
Vacuum cleaned Trash 2 in room right
Step 2 :
Vacuum decided to move to left from room right
Step 3 :
Vacuum decided to move to right from room left
Added a Trash at right
Step 4 :
Vacuum cleaned Trash 1 in room right
Added a Trash at left
Step 5 :
Vacuum decided to move to left from room right
Added a Trash at left
Step 6 :
Vacuum cleaned Trash 1 in room left
Vacuum cleaned Trash 2 in room left
Step 7 :
Vacuum decided to move to right from room left
Step 8 :
Vacuum decided to move to left from room right
Added a Trash at right
Step 9 :
Vacuum decided to move to right from room left
Added a Trash at right
